In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import optuna

import seaborn as sns

In [ ]:
# データの読み込み
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train_=df_train
df_test_=df_test

df_train_["train_test"]="train"
df_test_["train_test"]="test"

df_test_.insert(1,"Survived",np.nan)

df_all=pd.concat([df_train_,df_test_])

df_all.reset_index(drop=True)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train_test
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,test
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,test
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,test
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,test


In [ ]:
df_all["honor"]=df_all["Name"].map(lambda x:x.split(",")[1].split(".")[0])
df_all["honor"]=df_all["honor"].replace((" Don"," Rev"," Jonkheer"," Capt"),1)
df_all["honor"]=df_all["honor"].replace((" Mr"),2)
df_all["honor"]=df_all["honor"].replace((" Master"," Dr"," Major"," Col"),3)
df_all["honor"]=df_all["honor"].replace((" Miss",),4)
df_all["honor"]=df_all["honor"].replace((" Mrs"," Mme"," Ms"," Lady"," Sir"," Mlle"," the Countess"," Dona"),5)
df_all["Namelen"]=df_all["Name"].map(lambda x: len(str(x)))
df_all["Nameini"]=df_all["Name"].map(lambda x: str(x)[0])

In [ ]:
df_all.groupby("honor")["Age"].median()
df_all.loc[(df_all["honor"]==1)&(df_all["Age"].isnull()),"Age"]=41.0
df_all.loc[(df_all["honor"]==2)&(df_all["Age"].isnull()),"Age"]=29.0
df_all.loc[(df_all["honor"]==3)&(df_all["Age"].isnull()),"Age"]=7.0
df_all.loc[(df_all["honor"]==4)&(df_all["Age"].isnull()),"Age"]=22.0
df_all.loc[(df_all["honor"]==5)&(df_all["Age"].isnull()),"Age"]=35.0




df_all["Family"]=df_all["Parch"]+df_all["SibSp"]+1


df_all["Ticketini"]=df_all["Ticket"].map(lambda x: str(x)[0])
df_all["Ticketlen"]=df_all["Ticket"].map(lambda x: len(str(x)))


df_all["Fare"]=df_all["Fare"].fillna(df_all["Fare"].median())

df_all["Embarked"]=df_all["Embarked"].fillna("S")

In [ ]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Sex          1309 non-null   int64  
 4   Age          1309 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Fare         1309 non-null   float64
 8   Embarked     1309 non-null   int64  
 9   train_test   1309 non-null   object 
 10  honor        1309 non-null   int64  
 11  Namelen      1309 non-null   int64  
 12  Nameini      1309 non-null   int64  
 13  Family       1309 non-null   int64  
 14  Ticketini    1309 non-null   int64  
 15  Ticketlen    1309 non-null   int64  
dtypes: float64(3), int64(12), object(1)
memory usage: 173.9+ KB


In [ ]:
from sklearn.preprocessing import LabelEncoder
lbl=LabelEncoder()
df_all["Sex"]=lbl.fit_transform(df_all["Sex"])
df_all["Embarked"]=lbl.fit_transform(df_all["Embarked"])
df_all["Ticketini"]=lbl.fit_transform(df_all["Ticketini"])
df_all["Nameini"]=lbl.fit_transform(df_all["Nameini"])

In [ ]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Sex          1309 non-null   int64  
 4   Age          1309 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Fare         1309 non-null   float64
 8   Embarked     1309 non-null   int64  
 9   train_test   1309 non-null   object 
 10  honor        1309 non-null   int64  
 11  Namelen      1309 non-null   int64  
 12  Nameini      1309 non-null   int64  
 13  Family       1309 non-null   int64  
 14  Ticketini    1309 non-null   int64  
 15  Ticketlen    1309 non-null   int64  
dtypes: float64(3), int64(12), object(1)
memory usage: 173.9+ KB


In [ ]:
train=df_all[df_all["train_test"]=="train"]
testsub=df_all[df_all["train_test"]=="test"]

X=train.drop(["PassengerId","Survived","train_test"],axis=1)
y=train.iloc[:,1:2]
test=testsub.drop(["PassengerId","Survived","train_test"],axis=1)

In [ ]:
print(test.columns)

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'honor',
       'Namelen', 'Nameini', 'Family', 'Ticketini', 'Ticketlen'],
      dtype='object')


In [ ]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Sex          1309 non-null   int64  
 4   Age          1309 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Fare         1309 non-null   float64
 8   Embarked     1309 non-null   int64  
 9   train_test   1309 non-null   object 
 10  honor        1309 non-null   int64  
 11  Namelen      1309 non-null   int64  
 12  Nameini      1309 non-null   int64  
 13  Family       1309 non-null   int64  
 14  Ticketini    1309 non-null   int64  
 15  Ticketlen    1309 non-null   int64  
dtypes: float64(3), int64(12), object(1)
memory usage: 173.9+ KB


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# TensorFlowモデルの作成
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# モデルのコンパイル
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Early Stoppingのコールバックを設定
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# モデルの訓練
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,
                    batch_size=32,
                    callbacks=[early_stopping])

# バリデーションデータでの精度評価
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_accuracy:.4f}')

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5726 - loss: 1.0762 - val_accuracy: 0.6201 - val_loss: 0.6873
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6317 - loss: 0.7196 - val_accuracy: 0.6369 - val_loss: 0.6977
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6890 - loss: 0.6375 - val_accuracy: 0.6983 - val_loss: 0.6194
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7625 - loss: 0.5454 - val_accuracy: 0.7374 - val_loss: 0.5667
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7061 - loss: 0.6558 - val_accuracy: 0.7486 - val_loss: 0.5655
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7723 - loss: 0.5126 - val_accuracy: 0.7933 - val_loss: 0.5056
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7704 - loss: 0.4870 - val_accuracy: 0.7821 - val_loss: 0.5558
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7865 - loss: 0.4995 - val_accuracy: 0.7933 - val_loss: 0.

In [ ]:
import pandas as pd

# テストデータの予測結果を取得
predictions = model.predict(test)
predictions = (predictions > 0.5).astype(int).flatten()

# 提出用のデータフレームを作成
submission = pd.DataFrame({
    "PassengerId": testsub["PassengerId"],
    "Survived": predictions
})

# CSVファイルとして保存
submission.to_csv("submission.csv", index=False)

# 完了メッセージ
print("Submission file 'submission.csv' has been created and is ready for upload to Kaggle.")


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Submission file 'submission.csv' has been created and is ready for upload to Kaggle.
